# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [29]:
cities_df = pd.read_csv("weatherdata.csv")
gmaps.configure(api_key=g_key)
locations = cities_df[["Latitude","Longitude"]]
humidity = cities_df["Humidity"]
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=np.max(humidity), point_radius=3)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [16]:
#Find Cities that have a max temperature between 70 and 80 degrees, wind speed less than 10 mph and zero cloudiness
#Note that the temperatures in the dataframe are in Kelvin so the conditional statement uses the Kelvin equivalents of 70 and 80 degrees Fahrenheit
vacation_df = cities_df.loc[(cities_df['MaxTemp'] > 294) & (cities_df["MaxTemp"] < 300) & (cities_df["WindSpeed"] < 10) & (cities_df["Cloudiness"] == 0)]
vacation_df.reset_index(inplace=True)
del vacation_df["index"]
del vacation_df["Unnamed: 0"]
vacation_df

,City,Latitude,Longitude,MaxTemp,Cloudiness,Humidity,WindSpeed,Date
0,Saint-Philippe,-21.3585,55.7679,297.15,0,78,3.09,RE
1,Sekoma,-24.4000,23.8833,295.32,0,46,3.63,BW
2,Richards Bay,-28.7830,32.0377,296.98,0,82,6.02,ZA
3,Saint-Joseph,-21.3667,55.6167,297.15,0,78,3.09,RE
4,Yārāda,17.6500,83.2667,297.39,0,62,1.95,IN
5,Coyhaique,-45.5752,-72.0662,296.15,0,33,2.57,CL
6,Dubai,25.2582,55.3047,295.15,0,73,2.06,AE
7,Salalah,17.0151,54.0924,297.15,0,73,2.06,OM
8,Dzaoudzi,-12.7887,45.2699,299.15,0,89,2.57,YT
9,Pisco,-13.7000,-76.2167,298.15,0,73,3.09,PE


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [24]:
hotels = []

for i in range(len(vacation_df)):
    latitude = vacation_df.loc["Latitude"]
    longitude = vacation_df.loc["Longitude"]
    params = {
            "location": f"{latitude},{longitude}",
            "key": g_key,
            "radius": 5000,
            "types": "hotel",
    }
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response = requests.get(url, params=params)
    jsonfile = response.json()
    hotels.append(jsonfile)
    vacation_df["Hotel Name"] = hotels
    hotel_df = vacation_df.dropna(how='any')
vacation_df

KeyError: 'Latitude'

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
